# Capstone Project - The Battle of Neighborhoods, Final Report 
## Title: Predicting largest immigrant ethnic groups from neighborhood restaurant data

## Overview
* Author: Patrick Dolloso
* Date: Jan 19 2019
* Github repository location: https://github.com/patrickdolloso/IBM-DataScience-Capstone/blob/master/Capstone%20Project:%20Report.ipynb

## Introduction/Business Problem

### Introduction
With the rise of globalization, many major cities are seeing a rise of immigration from all over the world. Economic centres such as New York, Toronto, London, etc. are known for being multicultural hubs and home to many world-class enthic/fusion restaurants. 

### Problem Definition
Can we predict a multicultural city's major ethnic groups based on its most common restaurant categories?

### Business application
By seeing if there is a correlation that exists between the major types of ethnic restaurants in each borough/neighborhood and the city's largest ethnic groups -many valuable applications can be used with this data, for example: tailoring a restaurant supplier or grocery store's inventory to accommodate the tastes of different neighborhoods.

### Possible stakeholders/target audience
* Suppliers of restaurants
* Supermarket chains/grocery stores
* Importers/Exporters
* Prospective or landed immigrants
* General businesses/companies

## Data

### Cities to be analyzed
* Toronto: as the capital city of the province of Ontario, and the largest and most populous city in Canada -Toronto is known globally to be one of the most multicultural cities, a major economic hub, as well as one of the world's newest cities (incorporated as "Toronto" on March 6, 1834). Toronto: https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M

### About the data (and examples)
The data will consist of:
* Neighborhoods or boroughs and their locations, for example: latitude, longitute, address
* Restaurants and their ethnic categories, for example: Chinese, Italian, Indian, etc.
* Immigration data from government sources (for reference), for example: immigration numbers, ethnic backgrounds, etc.

### Methodology to execute and solve problem
1. Firstly, the scraped Neighborhood/Borough names will be stored in a pandas dataframe
2. Using the Foursquare API, the neighborhood locations (latitude, longitude) and the radius will be defined, which will produce a json data format. Also, 100 restaurants will be taken from the json data
3. The restaurant categories will be extracted from the json data and added into the pandas dataframe
4. The categories will be clustered into various ethnic groups (sushi = japanese, pizza = italian, etc.)
5. The venues will be tabulated and a prediction of the largest ethnic groups by neighborhood will be produced
6. This data will be compared to published data to predict largest immigrant ethnic group to see if restaurant categories have any influence on the ethnic groups present

#### TORONTO MAP: ETHNIC POPULATIONS IN TORONTO'S FOUR COMMUNITY COUNCILS
(reference: http://ureachtoronto.com/content/toronto-map-ethnic-populations-torontos-four-community-councils)

#### Toronto has 4 community councils:
* Toronto and East York
* North York
* Scarborough
* Etobicoke

In [165]:
from IPython.display import Image
from IPython.core.display import HTML 
Image(url= "https://patrickdolloso.files.wordpress.com/2019/01/1.png")

### Results
#### Toronto

In [1]:
# import libraries
import numpy as np
import pandas as pd
import msgpack
import json
import requests
from bs4 import BeautifulSoup
import csv
from pandas.io.json import json_normalize
import matplotlib.cm as cm
import matplotlib.colors as colors
from sklearn.cluster import KMeans
!pip install folium
import folium
!conda install -c conda-forge geopy --yes
from geopy.geocoders import Nominatim
!conda install -c conda-forge geocoder -y
!pip install geocoder
import geocoder
print('done.')

    100% |████████████████████████████████| 92kB 8.3MB/s eta 0:00:01
Requirement not upgraded as not directly required: requests in /opt/conda/envs/DSX-Python35/lib/python3.5/site-packages (from folium)
Requirement not upgraded as not directly required: jinja2 in /opt/conda/envs/DSX-Python35/lib/python3.5/site-packages (from folium)
Requirement not upgraded as not directly required: numpy in /opt/conda/envs/DSX-Python35/lib/python3.5/site-packages (from folium)
Requirement not upgraded as not directly required: six in /opt/conda/envs/DSX-Python35/lib/python3.5/site-packages (from folium)
Requirement not upgraded as not directly required: chardet<3.1.0,>=3.0.2 in /opt/conda/envs/DSX-Python35/lib/python3.5/site-packages (from requests->folium)
Requirement not upgraded as not directly required: idna<2.7,>=2.5 in /opt/conda/envs/DSX-Python35/lib/python3.5/site-packages (from requests->folium)
Requirement not upgraded as not directly required: urllib3<1.23,>=1.21.1 in /opt/conda/envs/DSX-Py

In [2]:
# Scrape toronto wiki data
toronto_link = requests.get('https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M')
toronto_soup = BeautifulSoup(toronto_link.text,'lxml')

# clean and assign into pandas dataframe
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None)
csv_file=open('toronto_1.csv','w')
csv_write=csv.writer(csv_file)
csv_write.writerow(['PostalCode','Borough','Neighborhood'])
for tr in toronto_soup.find_all('tr')[1:]:
    td_all = tr.find_all('td')
    if len(td_all) == 3:
        PostalCode = td_all[0].text
        Borough = td_all[1].text
        Neighborhood = td_all[2].text
    csv_write.writerow([PostalCode, Borough, Neighborhood])
csv_file.close()
df_toronto = pd.read_csv('toronto_1.csv')

#preview
df_toronto.head()

,PostalCode,Borough,Neighborhood
0,M1A,Not assigned,Not assigned\n
1,M2A,Not assigned,Not assigned\n
2,M3A,North York,Parkwoods\n
3,M4A,North York,Victoria Village\n
4,M5A,Downtown Toronto,Harbourfront\n


In [3]:
# remove \n enters
df_toronto[df_toronto.columns] =df_toronto.apply(lambda x: x.str.strip('\n'))
# remove all rows with 'Not assigned' Boroughs
df_toronto_cleaned = df_toronto[df_toronto.Borough!='Not assigned']
#preview
df_toronto_cleaned

,PostalCode,Borough,Neighborhood
2,M3A,North York,Parkwoods
3,M4A,North York,Victoria Village
4,M5A,Downtown Toronto,Harbourfront
5,M5A,Downtown Toronto,Regent Park
6,M6A,North York,Lawrence Heights
7,M6A,North York,Lawrence Manor
8,M7A,Queen's Park,Not assigned
10,M9A,Etobicoke,Islington Avenue
11,M1B,Scarborough,Rouge
12,M1B,Scarborough,Malvern


In [4]:
# reset index
df_toronto_cleaned=df_toronto_cleaned.reset_index()
df_toronto_cleaned.drop('index',axis=1,inplace=True)
df_toronto_cleaned

,PostalCode,Borough,Neighborhood
0,M3A,North York,Parkwoods
1,M4A,North York,Victoria Village
2,M5A,Downtown Toronto,Harbourfront
3,M5A,Downtown Toronto,Regent Park
4,M6A,North York,Lawrence Heights
5,M6A,North York,Lawrence Manor
6,M7A,Queen's Park,Not assigned
7,M9A,Etobicoke,Islington Avenue
8,M1B,Scarborough,Rouge
9,M1B,Scarborough,Malvern


In [85]:
# collect csv file of coordinates
link="http://cocl.us/Geospatial_data/Geospatial_Coordinates.csv"
df_toronto_coordinates=pd.read_csv(link)
df_toronto_coordinates.columns=['PostalCode','Latitude','Longitude']

# merge into a pandas dataframe
df_toronto_coordinates = pd.merge(df_toronto_cleaned, df_toronto_coordinates, on="PostalCode")
# Preview
df_toronto_coordinates

,PostalCode,Borough,Neighborhood,Latitude,Longitude
0,M3A,North York,Parkwoods,43.753259,-79.329656
1,M4A,North York,Victoria Village,43.725882,-79.315572
2,M5A,Downtown Toronto,Harbourfront,43.654260,-79.360636
3,M5A,Downtown Toronto,Regent Park,43.654260,-79.360636
4,M6A,North York,Lawrence Heights,43.718518,-79.464763
5,M6A,North York,Lawrence Manor,43.718518,-79.464763
6,M7A,Queen's Park,Not assigned,43.662301,-79.389494
7,M9A,Etobicoke,Islington Avenue,43.667856,-79.532242
8,M1B,Scarborough,Rouge,43.806686,-79.194353
9,M1B,Scarborough,Malvern,43.806686,-79.194353


## Results (Toronto)
### Data mine top restaurants in each borough
* Toronto has 11 Boroughs
* To simplify this process, we will find the top 5 ethnicities of 4 boroughs: Toronto, North York, Scarborough, Etobicoke.
### Toronto:

In [86]:
address = 'Toronto' #apply only to neighborhoods in Toronto
geolocator = Nominatim()
location = geolocator.geocode(address)
latitude = location.latitude
longitude = location.longitude
print('The geographical coordinates of {} are {}, {}'.format(address, latitude, longitude))

/opt/conda/envs/DSX-Python35/lib/python3.5/site-packages/ipykernel/__main__.py:2: DeprecationWarning: Using Nominatim with the default "geopy/1.18.1" `user_agent` is strongly discouraged, as it violates Nominatim's ToS https://operations.osmfoundation.org/policies/nominatim/ and may possibly cause 403 and 429 HTTP errors. Please specify a custom `user_agent` with `Nominatim(user_agent="my-application")` or by overriding the default `user_agent`: `geopy.geocoders.options.default_user_agent = "my-application"`. In geopy 2.0 this will become an exception.
  from ipykernel import kernelapp as app


The geographical coordinates of Toronto are 43.653963, -79.387207


In [87]:
# Find the number of boroughs and neighborhoods
print('{} has {} boroughs and {} neighborhoods'.format(
    address, len(df_toronto_coordinates['Borough'].unique()),
    df_toronto_coordinates.shape[0]
))

Toronto has 11 boroughs and 212 neighborhoods


In [88]:
# create a map to visualize the neighborhoods and boroughs
map_toronto = folium.Map(location=[latitude, longitude], zoom_start=12)

for lat, lng, borough, neighborhood in zip(df_toronto_coordinates['Latitude'], df_toronto_coordinates['Longitude'], 
df_toronto_coordinates['Borough'], df_toronto_coordinates['Neighborhood']):
    label = '{}, {}'.format(neighborhood, borough)
    label = folium.Popup(label, parse_html=True)
    folium.CircleMarker(
        [lat, lng],
        radius=5,
        popup=label,
        color='red',
        fill=True,
        fill_color='#3186cc',
        fill_opacity=0.7).add_to(map_toronto)

In [89]:
# preview map
map_toronto

### Input credentials in a hidden cell

In [90]:
# The code was removed by Watson Studio for sharing.

In [91]:
VERSION = '20180605' # Foursquare API version
LIMIT = 100
radius = 1000
search_query = 'Restaurant'

url = 'https://api.foursquare.com/v2/venues/search?client_id={}&client_secret={}&ll={},{}&v={}&query={}&radius={}&limit={}'.format(CLIENT_ID, CLIENT_SECRET, latitude, longitude, VERSION, search_query, radius, LIMIT)
    
toronto_results = requests.get(url).json()
toronto_results

{'meta': {'code': 200, 'requestId': '5c4a4d8add5797192a96be7b'},
 'response': {'venues': [{'categories': [{'icon': {'prefix': 'https://ss3.4sqi.net/img/categories_v2/food/default_',
       'suffix': '.png'},
      'id': '4bf58dd8d48988d14e941735',
      'name': 'American Restaurant',
      'pluralName': 'American Restaurants',
      'primary': True,
      'shortName': 'American'}],
    'hasPerk': False,
    'id': '4ad4c05ff964a52048f720e3',
    'location': {'address': '110 Chestnut Street',
     'cc': 'CA',
     'city': 'Toronto',
     'country': 'Canada',
     'distance': 145,
     'formattedAddress': ['110 Chestnut Street',
      'Toronto ON M5G 1R3',
      'Canada'],
     'labeledLatLngs': [{'label': 'display',
       'lat': 43.65488413420439,
       'lng': -79.38593077371578}],
     'lat': 43.65488413420439,
     'lng': -79.38593077371578,
     'postalCode': 'M5G 1R3',
     'state': 'ON'},
    'name': 'Hemispheres Restaurant & Bistro',
    'referralId': 'v-1548373386'},
   {'catego

In [92]:
from pandas.io.json import json_normalize

s = toronto_results['response']['venues']
json_normalize(s)
s

[{'categories': [{'icon': {'prefix': 'https://ss3.4sqi.net/img/categories_v2/food/default_',
     'suffix': '.png'},
    'id': '4bf58dd8d48988d14e941735',
    'name': 'American Restaurant',
    'pluralName': 'American Restaurants',
    'primary': True,
    'shortName': 'American'}],
  'hasPerk': False,
  'id': '4ad4c05ff964a52048f720e3',
  'location': {'address': '110 Chestnut Street',
   'cc': 'CA',
   'city': 'Toronto',
   'country': 'Canada',
   'distance': 145,
   'formattedAddress': ['110 Chestnut Street', 'Toronto ON M5G 1R3', 'Canada'],
   'labeledLatLngs': [{'label': 'display',
     'lat': 43.65488413420439,
     'lng': -79.38593077371578}],
   'lat': 43.65488413420439,
   'lng': -79.38593077371578,
   'postalCode': 'M5G 1R3',
   'state': 'ON'},
  'name': 'Hemispheres Restaurant & Bistro',
  'referralId': 'v-1548373386'},
 {'categories': [{'icon': {'prefix': 'https://ss3.4sqi.net/img/categories_v2/food/default_',
     'suffix': '.png'},
    'id': '4bf58dd8d48988d1c4941735',
   

In [93]:
# figure out how to access the category name
s[0]['categories']

[{'icon': {'prefix': 'https://ss3.4sqi.net/img/categories_v2/food/default_',
   'suffix': '.png'},
  'id': '4bf58dd8d48988d14e941735',
  'name': 'American Restaurant',
  'pluralName': 'American Restaurants',
  'primary': True,
  'shortName': 'American'}]

In [94]:
# loop through the JSON to pull the categories
for i in range(0,50):
    s[i]['categories']
    print(s[i]['categories'][0]['name'])

American Restaurant
Restaurant
Dim Sum Restaurant
Chinese Restaurant
Noodle House
Restaurant
Dim Sum Restaurant
Chinese Restaurant
Indian Restaurant
Restaurant
Bar
Restaurant
Chinese Restaurant
Caribbean Restaurant
Chinese Restaurant
Chinese Restaurant
Restaurant
Chinese Restaurant
Korean Restaurant
Sushi Restaurant
Breakfast Spot
Breakfast Spot
Restaurant
Indian Restaurant
Diner
Dim Sum Restaurant
Restaurant
Dim Sum Restaurant
Chinese Restaurant
Indian Restaurant
Japanese Restaurant
Chinese Restaurant
Chinese Restaurant
Vietnamese Restaurant
Bar
Modern European Restaurant
Restaurant
Sushi Restaurant
Thai Restaurant
French Restaurant
Restaurant
Theme Restaurant
Vietnamese Restaurant
Pub
English Restaurant
Vietnamese Restaurant
New American Restaurant
Restaurant
Greek Restaurant
Italian Restaurant


### North York, Toronto

In [95]:
address = 'North York, Ontario' #apply only to neighborhoods in Toronto
geolocator = Nominatim()
location = geolocator.geocode(address)
latitude = location.latitude
longitude = location.longitude
print('The geographical coordinates of {} are {}, {}'.format(address, latitude, longitude))

/opt/conda/envs/DSX-Python35/lib/python3.5/site-packages/ipykernel/__main__.py:2: DeprecationWarning: Using Nominatim with the default "geopy/1.18.1" `user_agent` is strongly discouraged, as it violates Nominatim's ToS https://operations.osmfoundation.org/policies/nominatim/ and may possibly cause 403 and 429 HTTP errors. Please specify a custom `user_agent` with `Nominatim(user_agent="my-application")` or by overriding the default `user_agent`: `geopy.geocoders.options.default_user_agent = "my-application"`. In geopy 2.0 this will become an exception.
  from ipykernel import kernelapp as app


The geographical coordinates of North York, Ontario are 43.7709163, -79.4124102


In [98]:
# create a map to visualize the neighborhoods and boroughs
map_NorthYork = folium.Map(location=[latitude, longitude], zoom_start=12)

for lat, lng, borough, neighborhood in zip(df_toronto_coordinates['Latitude'], df_toronto_coordinates['Longitude'], 
df_toronto_coordinates['Borough'], df_toronto_coordinates['Neighborhood']):
    label = '{}, {}'.format(neighborhood, borough)
    label = folium.Popup(label, parse_html=True)
    folium.CircleMarker(
        [lat, lng],
        radius=5,
        popup=label,
        color='red',
        fill=True,
        fill_color='#3186cc',
        fill_opacity=0.7).add_to(map_NorthYork)
    
#show map
map_NorthYork

In [99]:
VERSION = '20180605' # Foursquare API version
LIMIT = 100
radius = 1000
search_query = 'Restaurant'

url = 'https://api.foursquare.com/v2/venues/search?client_id={}&client_secret={}&ll={},{}&v={}&query={}&radius={}&limit={}'.format(CLIENT_ID, CLIENT_SECRET, latitude, longitude, VERSION, search_query, radius, LIMIT)
    
NorthYork_results = requests.get(url).json()
NorthYork_results

{'meta': {'code': 200, 'requestId': '5c4a6566dd5797192aff06b7'},
 'response': {'venues': [{'categories': [{'icon': {'prefix': 'https://ss3.4sqi.net/img/categories_v2/food/default_',
       'suffix': '.png'},
      'id': '4bf58dd8d48988d1c4941735',
      'name': 'Restaurant',
      'pluralName': 'Restaurants',
      'primary': True,
      'shortName': 'Restaurant'}],
    'hasPerk': False,
    'id': '537befcc498edb1da559269b',
    'location': {'address': '5221 Yonge St',
     'cc': 'CA',
     'city': 'Northyork',
     'country': 'Canada',
     'crossStreet': 'Parkview',
     'distance': 81,
     'formattedAddress': ['5221 Yonge St (Parkview)',
      'Northyork ON M2N 5P8',
      'Canada'],
     'labeledLatLngs': [{'label': 'display',
       'lat': 43.77107544003332,
       'lng': -79.41339624610163}],
     'lat': 43.77107544003332,
     'lng': -79.41339624610163,
     'postalCode': 'M2N 5P8',
     'state': 'ON'},
    'name': 'Symposium Cafe Restaurant & Lounge',
    'referralId': 'v-1548

In [100]:
from pandas.io.json import json_normalize

s = NorthYork_results['response']['venues']
json_normalize(s)
s

[{'categories': [{'icon': {'prefix': 'https://ss3.4sqi.net/img/categories_v2/food/default_',
     'suffix': '.png'},
    'id': '4bf58dd8d48988d1c4941735',
    'name': 'Restaurant',
    'pluralName': 'Restaurants',
    'primary': True,
    'shortName': 'Restaurant'}],
  'hasPerk': False,
  'id': '537befcc498edb1da559269b',
  'location': {'address': '5221 Yonge St',
   'cc': 'CA',
   'city': 'Northyork',
   'country': 'Canada',
   'crossStreet': 'Parkview',
   'distance': 81,
   'formattedAddress': ['5221 Yonge St (Parkview)',
    'Northyork ON M2N 5P8',
    'Canada'],
   'labeledLatLngs': [{'label': 'display',
     'lat': 43.77107544003332,
     'lng': -79.41339624610163}],
   'lat': 43.77107544003332,
   'lng': -79.41339624610163,
   'postalCode': 'M2N 5P8',
   'state': 'ON'},
  'name': 'Symposium Cafe Restaurant & Lounge',
  'referralId': 'v-1548379494',
  'venuePage': {'id': '146437887'}},
 {'categories': [{'icon': {'prefix': 'https://ss3.4sqi.net/img/categories_v2/food/pizza_',
    

In [103]:
# loop through the JSON to pull the categories
for i in range(0,17):
    s[i]['categories']
    print(s[i]['categories'][0]['name'])

Restaurant
Pizza Place
Chinese Restaurant
Chinese Restaurant
Asian Restaurant
Restaurant
Chinese Restaurant
Korean Restaurant
Diner
Korean Restaurant
Japanese Restaurant
Korean Restaurant
Chinese Restaurant
Korean Restaurant
Japanese Restaurant
Restaurant
Korean Restaurant


### Scarborough, Toronto

In [147]:
address = 'Scarborough, ON' # Scarborough
geolocator = Nominatim()
location = geolocator.geocode(address)
latitude = location.latitude
longitude = location.longitude
print('The geographical coordinates of {} are {}, {}'.format(address, latitude, longitude))

/opt/conda/envs/DSX-Python35/lib/python3.5/site-packages/ipykernel/__main__.py:2: DeprecationWarning: Using Nominatim with the default "geopy/1.18.1" `user_agent` is strongly discouraged, as it violates Nominatim's ToS https://operations.osmfoundation.org/policies/nominatim/ and may possibly cause 403 and 429 HTTP errors. Please specify a custom `user_agent` with `Nominatim(user_agent="my-application")` or by overriding the default `user_agent`: `geopy.geocoders.options.default_user_agent = "my-application"`. In geopy 2.0 this will become an exception.
  from ipykernel import kernelapp as app


The geographical coordinates of Scarborough, ON are 43.773077, -79.257774


In [148]:
# create a map to visualize the neighborhoods and boroughs
map_scarborough = folium.Map(location=[latitude, longitude], zoom_start=12)

for lat, lng, borough, neighborhood in zip(df_toronto_coordinates['Latitude'], df_toronto_coordinates['Longitude'], 
df_toronto_coordinates['Borough'], df_toronto_coordinates['Neighborhood']):
    label = '{}, {}'.format(neighborhood, borough)
    label = folium.Popup(label, parse_html=True)
    folium.CircleMarker(
        [lat, lng],
        radius=5,
        popup=label,
        color='red',
        fill=True,
        fill_color='#3186cc',
        fill_opacity=0.7).add_to(map_scarborough)
    
#show map
map_scarborough

In [149]:
VERSION = '20180605' # Foursquare API version
LIMIT = 100
radius = 1000
search_query = 'Restaurant'

url = 'https://api.foursquare.com/v2/venues/search?client_id={}&client_secret={}&ll={},{}&v={}&query={}&radius={}&limit={}'.format(CLIENT_ID, CLIENT_SECRET, latitude, longitude, VERSION, search_query, radius, LIMIT)
    
scarborough_results = requests.get(url).json()
scarborough_results

{'meta': {'code': 200, 'requestId': '5c4a692f4c1f674506921665'},
 'response': {'venues': [{'categories': [],
    'hasPerk': False,
    'id': '4bf712334a67c9286dfc22cf',
    'location': {'address': '2598 Birchmount Road',
     'cc': 'CA',
     'city': 'Scarborough',
     'country': 'Canada',
     'distance': 520,
     'formattedAddress': ['2598 Birchmount Road', 'Scarborough ON', 'Canada'],
     'labeledLatLngs': [{'label': 'display',
       'lat': 43.777092430965595,
       'lng': -79.25446174597401}],
     'lat': 43.777092430965595,
     'lng': -79.25446174597401,
     'state': 'ON'},
    'name': 'Fortune House Restaurant',
    'referralId': 'v-1548380463'},
   {'categories': [{'icon': {'prefix': 'https://ss3.4sqi.net/img/categories_v2/food/caribbean_',
       'suffix': '.png'},
      'id': '4bf58dd8d48988d144941735',
      'name': 'Caribbean Restaurant',
      'pluralName': 'Caribbean Restaurants',
      'primary': True,
      'shortName': 'Caribbean'}],
    'hasPerk': False,
    'id

In [150]:
from pandas.io.json import json_normalize

s = scarborough_results['response']['venues']
json_normalize(s)
s

[{'categories': [],
  'hasPerk': False,
  'id': '4bf712334a67c9286dfc22cf',
  'location': {'address': '2598 Birchmount Road',
   'cc': 'CA',
   'city': 'Scarborough',
   'country': 'Canada',
   'distance': 520,
   'formattedAddress': ['2598 Birchmount Road', 'Scarborough ON', 'Canada'],
   'labeledLatLngs': [{'label': 'display',
     'lat': 43.777092430965595,
     'lng': -79.25446174597401}],
   'lat': 43.777092430965595,
   'lng': -79.25446174597401,
   'state': 'ON'},
  'name': 'Fortune House Restaurant',
  'referralId': 'v-1548380463'},
 {'categories': [{'icon': {'prefix': 'https://ss3.4sqi.net/img/categories_v2/food/caribbean_',
     'suffix': '.png'},
    'id': '4bf58dd8d48988d144941735',
    'name': 'Caribbean Restaurant',
    'pluralName': 'Caribbean Restaurants',
    'primary': True,
    'shortName': 'Caribbean'}],
  'hasPerk': False,
  'id': '51a92c08011cd19fbae73072',
  'location': {'address': '1249 Ellesmere Rd',
   'cc': 'CA',
   'city': 'Toronto',
   'country': 'Canada',


In [153]:
# loop through the JSON to pull the categories
for i in range(0,5):
    s[i]
    print(s[i]['name'])

Fortune House Restaurant
Supreme Restaurant and Bar
Canyon Creek Chophouse
The Blue Danube
Cora's


### Etobicoke

In [154]:
address = 'Etobicoke' # Scarborough
geolocator = Nominatim()
location = geolocator.geocode(address)
latitude = location.latitude
longitude = location.longitude
print('The geographical coordinates of {} are {}, {}'.format(address, latitude, longitude))

/opt/conda/envs/DSX-Python35/lib/python3.5/site-packages/ipykernel/__main__.py:2: DeprecationWarning: Using Nominatim with the default "geopy/1.18.1" `user_agent` is strongly discouraged, as it violates Nominatim's ToS https://operations.osmfoundation.org/policies/nominatim/ and may possibly cause 403 and 429 HTTP errors. Please specify a custom `user_agent` with `Nominatim(user_agent="my-application")` or by overriding the default `user_agent`: `geopy.geocoders.options.default_user_agent = "my-application"`. In geopy 2.0 this will become an exception.
  from ipykernel import kernelapp as app


The geographical coordinates of Etobicoke are 43.6435559, -79.5656326


In [156]:
# create a map to visualize the neighborhoods and boroughs
map_etobicoke = folium.Map(location=[latitude, longitude], zoom_start=12)

for lat, lng, borough, neighborhood in zip(df_toronto_coordinates['Latitude'], df_toronto_coordinates['Longitude'], 
df_toronto_coordinates['Borough'], df_toronto_coordinates['Neighborhood']):
    label = '{}, {}'.format(neighborhood, borough)
    label = folium.Popup(label, parse_html=True)
    folium.CircleMarker(
        [lat, lng],
        radius=5,
        popup=label,
        color='red',
        fill=True,
        fill_color='#3186cc',
        fill_opacity=0.7).add_to(map_etobicoke)
    
#show map
map_etobicoke

In [157]:
VERSION = '20180605' # Foursquare API version
LIMIT = 100
radius = 1000
search_query = 'Restaurant'

url = 'https://api.foursquare.com/v2/venues/search?client_id={}&client_secret={}&ll={},{}&v={}&query={}&radius={}&limit={}'.format(CLIENT_ID, CLIENT_SECRET, latitude, longitude, VERSION, search_query, radius, LIMIT)
    
etobicoke_results = requests.get(url).json()
etobicoke_results

{'meta': {'code': 200, 'requestId': '5c4a69926a60717af388e71e'},
 'response': {'venues': [{'categories': [{'icon': {'prefix': 'https://ss3.4sqi.net/img/categories_v2/food/default_',
       'suffix': '.png'},
      'id': '4bf58dd8d48988d1c4941735',
      'name': 'Restaurant',
      'pluralName': 'Restaurants',
      'primary': True,
      'shortName': 'Restaurant'}],
    'hasPerk': False,
    'id': '534b2c4c498e3290a42bc2f1',
    'location': {'address': '555 Burnhamthorpe',
     'cc': 'CA',
     'city': 'Etobicoke',
     'country': 'Canada',
     'crossStreet': 'The West Mall',
     'distance': 231,
     'formattedAddress': ['555 Burnhamthorpe (The West Mall)',
      'Etobicoke ON M9C 2Y3',
      'Canada'],
     'labeledLatLngs': [{'label': 'display',
       'lat': 43.64472458866585,
       'lng': -79.56800937652586}],
     'lat': 43.64472458866585,
     'lng': -79.56800937652586,
     'postalCode': 'M9C 2Y3',
     'state': 'ON'},
    'name': 'Cross Eyed Bear Restaurant',
    'referralI

In [158]:
from pandas.io.json import json_normalize

s = etobicoke_results['response']['venues']
json_normalize(s)
s

[{'categories': [{'icon': {'prefix': 'https://ss3.4sqi.net/img/categories_v2/food/default_',
     'suffix': '.png'},
    'id': '4bf58dd8d48988d1c4941735',
    'name': 'Restaurant',
    'pluralName': 'Restaurants',
    'primary': True,
    'shortName': 'Restaurant'}],
  'hasPerk': False,
  'id': '534b2c4c498e3290a42bc2f1',
  'location': {'address': '555 Burnhamthorpe',
   'cc': 'CA',
   'city': 'Etobicoke',
   'country': 'Canada',
   'crossStreet': 'The West Mall',
   'distance': 231,
   'formattedAddress': ['555 Burnhamthorpe (The West Mall)',
    'Etobicoke ON M9C 2Y3',
    'Canada'],
   'labeledLatLngs': [{'label': 'display',
     'lat': 43.64472458866585,
     'lng': -79.56800937652586}],
   'lat': 43.64472458866585,
   'lng': -79.56800937652586,
   'postalCode': 'M9C 2Y3',
   'state': 'ON'},
  'name': 'Cross Eyed Bear Restaurant',
  'referralId': 'v-1548380562'},
 {'categories': [{'icon': {'prefix': 'https://ss3.4sqi.net/img/categories_v2/food/scandinavian_',
     'suffix': '.png'}

In [161]:
# loop through the JSON to pull the categories
for i in range(0,2):
    s[i]['categories']
    print(s[i]['categories'][0]['name'])

Restaurant
Scandinavian Restaurant


## Discussion

### Analysis of the Results
* The following image shows the top ethnic groups in Toronto


In [2]:
from IPython.display import Image
from IPython.core.display import HTML 
Image(url= "https://canadaimmigrants.com/wp-content/uploads/2016/03/Toronto-Immigrants-w-600x479.png")

As we can see the top 3 immigrants are from India, China, and the Philippines.
Although the analysis accurately predicted Chinese and Indian ethnicities, filipino and Italian restaurants are underrepresented.
And Korean and Vietnamese are Over-represented.